# Réunir les fichiers en un seul DataFrame

In [1]:
from pandas import DataFrame, read_json
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import csv
import os

## Pour les publications
Je concatene les différents fichiers obtenus à l'aide du scrapping.

In [2]:
json_files = [pos_json for pos_json in os.listdir() if pos_json.startswith('insta_publi')]

insta_publi_final=pd.DataFrame()

for index, js in enumerate(json_files):
    with open(os.path.join(js)) as json_file:
        insta_publi_final = insta_publi_final.append(pd.read_json(js),sort=False)

In [3]:
insta_publi_final.tail(3)

,code_jour_semaine,date,date_time,day,hashtag,hashtag_count,month,node.__typename,node.accessibility_caption,node.comments_disabled,...,node.thumbnail_src,node.video_view_count,text_publication,text_publication_temp,time,year,node.felix_profile_grid_crop.crop_bottom,node.felix_profile_grid_crop.crop_left,node.felix_profile_grid_crop.crop_right,node.felix_profile_grid_crop.crop_top
1733,7,2019-01-13,2019-01-13 23:10:12,13,"[adolescence.....#nouslesdeviants, #skuse, #ve...",15,1,GraphImage,L’image contient peut-être : texte,False,...,https://scontent-cdt1-1.cdninstagram.com/vp/ce...,NaN,"Quand Ella Newhall avait seize ans, elle appar...",[{'node.text': 'Quand Ella Newhall avait seize...,23:10:12,2019,NaN,NaN,NaN,NaN
1734,3,2019-01-09,2019-01-09 20:38:54,9,"[Buntin....#labellecolere, #labellecolère, #ma...",13,1,GraphImage,L’image contient peut-être : plein air,False,...,https://scontent-cdt1-1.cdninstagram.com/vp/8b...,NaN,"« Chaque fois que j’entends le mot danger, je ...",[{'node.text': '« Chaque fois que j’entends le...,20:38:54,2019,NaN,NaN,NaN,NaN
1735,4,2019-01-03,2019-01-03 14:42:39,3,"[colère.....#retrospective, #labellecolere, #l...",12,1,GraphImage,L’image contient peut-être : 2 personnes,False,...,https://scontent-cdt1-1.cdninstagram.com/vp/97...,NaN,Mets-moi dans une case et je te montrerai ma c...,[{'node.text': 'Mets-moi dans une case et je t...,14:42:39,2019,NaN,NaN,NaN,NaN


In [4]:
insta_publi_final.columns

Index(['code_jour_semaine', 'date', 'date_time', 'day', 'hashtag',
       'hashtag_count', 'month', 'node.__typename',
       'node.accessibility_caption', 'node.comments_disabled',
       'node.dimensions.height', 'node.dimensions.width', 'node.display_url',
       'node.edge_liked_by.count', 'node.edge_media_preview_like.count',
       'node.edge_media_to_caption.edges', 'node.edge_media_to_comment.count',
       'node.fact_check_information', 'node.fact_check_overall_rating',
       'node.felix_profile_grid_crop', 'node.gating_info', 'node.id',
       'node.is_video', 'node.location', 'node.location.has_public_page',
       'node.location.id', 'node.location.name', 'node.location.slug',
       'node.media_preview', 'node.owner.id', 'node.owner.username',
       'node.shortcode', 'node.taken_at_timestamp', 'node.thumbnail_resources',
       'node.thumbnail_src', 'node.video_view_count', 'text_publication',
       'text_publication_temp', 'time', 'year',
       'node.felix_profile_gri

# Déplacer et renommer les colonnes + supprimer les colonnes jugées inutiles

In [5]:
insta_publi_final = insta_publi_final[['node.owner.id', 'node.owner.username',
       'date_time', 'date','code_jour_semaine', 'day','month', 'year','time',
       'node.id','node.__typename', 
       'text_publication','hashtag','hashtag_count','node.edge_liked_by.count',
       'node.dimensions.height', 'node.dimensions.width', 'node.display_url',
       'node.edge_media_preview_like.count','node.edge_media_to_comment.count','node.video_view_count', 
       'node.edge_media_to_caption.edges', 
       'node.fact_check_information','node.gating_info',  'node.is_video', 'node.location',
       'node.location.has_public_page', 'node.location.id',
       'node.location.name', 'node.location.slug',
       'node.shortcode','node.thumbnail_src']]

Je renomme la colonne 'node.owner.id' en 'page_id' afin de pouvoir merger les deux dataframes que je vais obtenir.

In [6]:
insta_publi_final = insta_publi_final.rename(columns={'node.owner.id':'page_id'})

In [7]:
insta_publi_final.head(3)

,page_id,node.owner.username,date_time,date,code_jour_semaine,day,month,year,time,node.id,...,node.fact_check_information,node.gating_info,node.is_video,node.location,node.location.has_public_page,node.location.id,node.location.name,node.location.slug,node.shortcode,node.thumbnail_src
0,4741863243,librairielestempsmodernes,2017-03-09 22:16:32,2017-03-09,4,9,3,2017,22:16:32,1466950865096294144,...,NaN,NaN,False,NaN,NaN,NaN,None,None,BRbqAohBD9A,https://scontent-cdt1-1.cdninstagram.com/vp/5b...
1,4741863243,librairielestempsmodernes,2017-03-01 20:16:30,2017-03-01,3,1,3,2017,20:16:30,1461092250598961920,...,NaN,NaN,False,NaN,NaN,NaN,None,None,BRG16klBvN5,https://scontent-cdt1-1.cdninstagram.com/vp/bc...
2,3655964171,librairielibellule,2019-10-11 16:56:30,2019-10-11,5,11,10,2019,16:56:30,2152397435142459904,...,NaN,NaN,False,NaN,1.0,6.093318e+14,Librairie Libellule,librairie-libellule,B3e2hA5INJC,https://scontent-cdt1-1.cdninstagram.com/vp/cb...


## Export de la liste des publications

In [8]:
insta_publi_final.to_json('../CONCATENER/insta_publi_final_8_NOV_2019.json', orient='records')

## Pour les infos globales
Je concatene les différents fichiers obtenus à l'aide du scrapping.

In [9]:
json_files_info = [pos_json for pos_json in os.listdir() if pos_json.startswith('insta_info')]

insta_info_final=pd.DataFrame()

for index, js in enumerate(json_files_info):
    with open(os.path.join(js)) as json_file:
        insta_info_final = insta_info_final.append(pd.read_json(js),sort=False)

In [10]:
insta_info_final=insta_info_final.assign(page_id=insta_info_final.logging_page_id.apply(lambda x : x.split('_')))
insta_info_final.page_id = insta_info_final.page_id.map(lambda publi: publi[1]).apply(pd.to_numeric)
insta_info_final = insta_info_final.drop(columns='logging_page_id')

### Fusion des deux fichiers obtenus

In [11]:
insta_total = pd.merge(insta_publi_final, insta_info_final, on='page_id', how='left')
insta_total.head()

,page_id,node.owner.username,date_time,date,code_jour_semaine,day,month,year,time,node.id,...,node.location,node.location.has_public_page,node.location.id,node.location.name,node.location.slug,node.shortcode,node.thumbnail_src,graphql.user.edge_followed_by.count,graphql.user.edge_owner_to_timeline_media.count,graphql.user.username
0,4741863243,librairielestempsmodernes,2017-03-09 22:16:32,2017-03-09,4,9,3,2017,22:16:32,1466950865096294144,...,NaN,NaN,NaN,None,None,BRbqAohBD9A,https://scontent-cdt1-1.cdninstagram.com/vp/5b...,224,2,librairielestempsmodernes
1,4741863243,librairielestempsmodernes,2017-03-01 20:16:30,2017-03-01,3,1,3,2017,20:16:30,1461092250598961920,...,NaN,NaN,NaN,None,None,BRG16klBvN5,https://scontent-cdt1-1.cdninstagram.com/vp/bc...,224,2,librairielestempsmodernes
2,3655964171,librairielibellule,2019-10-11 16:56:30,2019-10-11,5,11,10,2019,16:56:30,2152397435142459904,...,NaN,1.0,6.093318e+14,Librairie Libellule,librairie-libellule,B3e2hA5INJC,https://scontent-cdt1-1.cdninstagram.com/vp/cb...,235,366,librairielibellule
3,3655964171,librairielibellule,2019-10-10 12:01:06,2019-10-10,4,10,10,2019,12:01:06,2151523975361193472,...,NaN,1.0,6.093318e+14,Librairie Libellule,librairie-libellule,B3bv6gFI-Wm,https://scontent-cdt1-1.cdninstagram.com/vp/f1...,235,366,librairielibellule
4,3655964171,librairielibellule,2019-08-31 18:55:53,2019-08-31,6,31,8,2019,18:55:53,2122741714331470080,...,NaN,1.0,6.093318e+14,Librairie Libellule,librairie-libellule,B11fleooYlZ,https://scontent-cdt1-1.cdninstagram.com/vp/4e...,235,366,librairielibellule


In [12]:
insta_total.to_json('../CONCATENER/insta_total_8_NOV_2019.json', orient='records')